# **(Healthcare Insurance Cost Analysis Assessment)**

## Objectives

* For this project i gathered the raw data from Kaggle(https://www.kaggle.com/datasets/willianoliveiragibin/healthcare-insurance/data). The objective of this notebook is to extract raw insurance data, apply a series of cleaning and transformation steps, and produce a validated, structured DataFrame suitable for analysis and visualization.
- Key goals include:
- Removing duplicates and irrelevant columns
- Handling missing values and inconsistent categories
- Engineering new features (e.g., BMI categories, age groups)



## Inputs

* We need the raw insurance data from Kaggle, as well as the necessary libraries for data manipulation and visualization (e.g., pandas, numpy, seaborn, plotly.express, matplotlib) as shown in section one.

i have included the raw data file in the project directory. 

## Outputs

- By the end of this notebook, the following artefacts are generated:
- Cleaned DataFrame: df_cleaned — ready for analysis and visualization
- Exported file: cleaned_data.csv saved to /dataset/cleaned_data/
- Validation report: summary of missing values, duplicates removed, and category standardization
- Feature engineering log: notes on new columns added (e.g., bmi_category, age_group)
 

## Additional Comments

* We found 1 duplicate entry in the dataset, which has been removed during the cleaning process. We also found an outlier which has been flagged in the code for review. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [239]:
import os
current_dir = os.getcwd()
current_dir

'c:\\'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [240]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [241]:
current_dir = os.getcwd()
current_dir

'c:\\'

# Section 1

Section 1 content

In [242]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



---

# Section 2

Section 2 content

In [243]:
# Load raw data
df = pd.read_csv("C:\\Users\\Nine\\OneDrive\\Documents\\VS Code Projects\\archive\\insurance.csv")
df.head()

print(df.head())



   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [244]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [245]:
#Check for missing values
df.isnull().sum()
print (df)

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


In [246]:
# Check for duplicates.
print(df.duplicated())

# Duplicates returned as false, To make sure we will normalise the data to make sure there are no additional spaces to make values seem different.


0       False
1       False
2       False
3       False
4       False
        ...  
1333    False
1334    False
1335    False
1336    False
1337    False
Length: 1338, dtype: bool


In [247]:
#here we are making all text lowercase and strip space
for col in df.select_dtypes(include='object'):
    df[col] = df[col].str.strip().str.lower()

In [248]:
print(df.duplicated())

0       False
1       False
2       False
3       False
4       False
        ...  
1333    False
1334    False
1335    False
1336    False
1337    False
Length: 1338, dtype: bool


In [249]:
#here we are round all numeric values to 2 decimal places
for col in df.select_dtypes(include='float'):
    df[col] = df[col].round(2)

In [250]:
print(df.duplicated().sum())

#here is now showing there is 1 duplicate

1


In [251]:
#here we will check where the duplicates are
df[df.duplicated(keep=False)]

,age,sex,bmi,children,smoker,region,charges
195,19,male,30.59,0,no,northwest,1639.56
581,19,male,30.59,0,no,northwest,1639.56


In [252]:
df[df.duplicated()]

,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.56


In [253]:
#We will now delete the duplicate

df = df.drop_duplicates()

In [254]:
print(df.duplicated().sum())

#now we can see that there are no more duplicates

0


In [255]:
# Convert categorical columns to category dtype
categorical_cols = ['sex', 'smoker', 'region']
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [256]:
df[['sex','smoker','region']].value_counts()

#This shows that we have no missing or mispelt values in the categorical columns

sex     smoker  region   
female  no      southwest    141
                southeast    139
                northwest    135
male    no      southeast    134
female  no      northeast    132
male    no      northwest    131
                southwest    126
                northeast    125
        yes     southeast     55
                northeast     38
                southwest     37
female  yes     southeast     36
                northeast     29
                northwest     29
male    yes     northwest     29
female  yes     southwest     21
Name: count, dtype: int64

In [257]:
df.describe()


,age,bmi,children,charges
count,1337.000000,1337.000000,1337.000000,1337.000000
mean,39.222139,30.663628,1.095737,13279.121503
std,14.044333,6.100233,1.205571,12110.359677
min,18.000000,15.960000,0.000000,1121.870000
25%,27.000000,26.290000,0.000000,4746.340000
50%,39.000000,30.400000,1.000000,9386.160000
75%,51.000000,34.700000,2.000000,16657.720000
max,64.000000,53.130000,5.000000,63770.430000


In [258]:
#The max BMI may be a mistake or incorrect input
df[df['bmi'] == df['bmi'].max()]


,age,sex,bmi,children,smoker,region,charges
1317,18,male,53.13,0,no,southeast,1163.46


In [259]:
df.sort_values(by='bmi', ascending=False).head(5)[['age','bmi','charges','smoker','region']]

# After checking these values it seems false as the BMI is high for an 18 year old, who doesn't smoke and is paying a low premium
# This may be an error in the data entry process


,age,bmi,charges,smoker,region
1317,18,53.13,1163.46,no,southeast
1047,22,52.58,44501.40,yes,southeast
847,23,50.38,2438.06,no,southeast
116,58,49.06,11381.33,no,southeast
286,46,48.07,9432.93,no,northeast


In [260]:
#I will flag the bmi as an outlier
df['is_bmi_outlier'] = df['bmi'] > 53


#Flagged outliers
df['is_bmi_outlier'].value_counts()

is_bmi_outlier
False    1336
True        1
Name: count, dtype: int64

In [261]:
#This will remove extreme BMI outliers from the dataset
#We can block out this block if we want to keep extreme values for analysis

before = len(df)
df = df[df['bmi'] <= 52].reset_index(drop=True)
after = len(df)
print(f"Removed {before - after} extreme BMI outliers")

Removed 2 extreme BMI outliers


In [262]:
# Create BMI category
def bmi_category(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif bmi < 25:
        return 'Normal'
    elif bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

df['bmi_category'] = df['bmi'].apply(bmi_category)

In [263]:
#Checking underweight category
df[df['bmi'] < 18.5][['bmi', 'bmi_category']].head()


,bmi,bmi_category
28,17.39,Underweight
128,17.76,Underweight
172,15.96,Underweight
198,18.05,Underweight
232,17.80,Underweight


In [264]:
# Create age category
def age_category(age):
    if age < 25:
        return '18-24'
    elif 25 <= age < 35:
        return '25-34'
    elif 35 <= age < 45:
        return '35-44'
    elif 45 <= age < 55:
        return '45-54'
    else:
        return '55+'

df['age_category'] = df['age'].apply(age_category)


In [265]:
# Checking age category
df[df['age'] < 25][['age', 'age_category']].head()

,age,age_category
0,19,18-24
1,18,18-24
12,23,18-24
15,19,18-24
17,23,18-24


In [266]:
# Smokers with children
smoker_children = df[(df['smoker'] == 'yes') & (df['children'] >= 1)]

print("Smokers with children (head):")
display(smoker_children[['sex','children','charges']].head())

print("\nAverage charge by sex among smokers with children:")
summary = smoker_children.groupby('sex', observed=True)['charges'].agg(['mean','count'])
display(summary)

Smokers with children (head):


,sex,children,charges
23,female,1,37701.88
29,male,2,38711.00
34,male,1,51194.56
38,male,1,39774.28
49,male,1,38709.18



Average charge by sex among smokers with children:


,mean,count
sex,,
female,30673.812742,62
male,33658.634271,96


In [267]:
# Non-smokers with children
nonsmoker_children = df[(df['smoker'] == 'no') & (df['children'] >= 1)]

print("Non-smokers with children (head):")
display(nonsmoker_children[['sex','children','charges']].head())

print("\nAverage charge by sex among non-smokers with children:")
summary2 = nonsmoker_children.groupby('sex', observed=True)['charges'].agg(['mean','count'])
display(summary2)

Non-smokers with children (head):


,sex,children,charges
1,male,1,1725.55
2,male,3,4449.46
6,female,1,8240.59
7,female,3,7281.51
8,male,2,6406.41



Average charge by sex among non-smokers with children:


,mean,count
sex,,
female,9577.277685,311
male,8509.234592,294


In [268]:
#Final check to make sure data is clean

print("Missing values per column:")
print(df.isna().sum())
print("\nShape of cleaned dataset:", df.shape)


Missing values per column:
age               0
sex               0
bmi               0
children          0
smoker            0
region            0
charges           0
is_bmi_outlier    0
bmi_category      0
age_category      0
dtype: int64

Shape of cleaned dataset: (1335, 10)


In [269]:
# Check to ensure memory usage is optimised
print("Memory usage (MB):")
print(df.memory_usage(deep=True).sum() / (1024**2))

Memory usage (MB):
0.1859607696533203


---

---